In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
Raw['tf'].unique()

In [ ]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['A']        = Raw['a']
    Src['Z']        = Raw['z']
    Src['Symbol']   = Raw['symbol']
    Src['TF']       = Raw['tf']
    
    Src['Datetime'] = Raw['datetime']
    Src['Date']     = Raw['datetime'].dt.date
    Src['Time']     = Raw['datetime'].dt.time
    
    Src['Ticks']    = Raw['tick_volume']
    Src['Volume']   = Raw['real_volume']
    Src['Price']    = Raw['close']
    
    Src['Open']     = Raw['open']
    Src['High']     = Raw['high']
    Src['Low']      = Raw['low']
    Src['Close']    = Raw['close']

    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)

    Src['Change']       = Src['Close'] - Src['Open']
    Src['HL']           = Src['High']  - Src['Low']
    return Src 

Src = FORMAT_SOURCE(Raw)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Psc']         = Calc.groupby(['Symbol','TF'], sort=0)['Change'] .expanding(20).rank(ascending=1, pct=1).mul(100)      .reset_index(drop=1)
    Calc['HL Psc']          = Calc.groupby(['Symbol','TF'], sort=0)['HL']     .expanding(20).rank(ascending=1, pct=1).mul(100)      .reset_index(drop=1)
    
    Calc['Chg Norm']        = Calc.apply(lambda x: x['Change'] if (5 < x['Chg Psc'] < 95) else nan, axis=1)
    Calc['HL Norm']         = Calc.apply(lambda x: x['HL']     if (5 < x['HL Psc']  < 95) else nan, axis=1)

    Calc['Chg Norm Abs']    = Calc['Chg Norm'].abs()
    Calc['Chg Norm Pos']    = Calc['Chg Norm'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Chg Norm Neg']    = Calc['Chg Norm'].apply(lambda x: x if (x <  0) else nan)
    
    Calc['HL Norm Pos']     = Calc['HL Norm'] * Calc['Bull Sign']
    Calc['HL Norm Neg']     = Calc['HL Norm'] * Calc['Bear Sign']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [15]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm']].describe().round(0).astype(int)

Chg Norm                                       
              count mean   std   min   25% 50%   75%   max
Symbol TF                                                 
WIN$N  M5     91498    0    78  -390   -50   0    50   470
       M15    42305   -1   142  -390   -95   0    90   380
       H1     10893   -6   291  -915  -190   0   190   730
       H3      4053   -9   492 -1410  -295   5   285  1225
       D1      1111  -53  1026 -2705  -820  10   700  2040
       W1       218   55  2145 -4780 -1544 -70  1880  4565

In [16]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Abs']].describe().round(0).astype(int)

Chg Norm Abs                                       
                  count  mean   std min  25%   50%   75%   max
Symbol TF                                                     
WIN$N  M5         91498    62    47   0   25    50    90   470
       M15        42305   113    87   0   45    90   165   390
       H1         10893   230   178   0   85   190   340   915
       H3          4053   380   312   0  120   290   575  1410
       D1          1111   849   578   0  385   765  1210  2705
       W1           218  1775  1200   0  799  1635  2528  4780

In [17]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Pos']].describe().round(0).astype(int)

Chg Norm Pos                                       
                  count  mean   std min  25%   50%   75%   max
Symbol TF                                                     
WIN$N  M5         47322    60    47   0   20    50    90   470
       M15        21504   110    86   0   40    90   165   380
       H1          5492   223   171   0   80   190   330   730
       H3          2066   363   299   0  115   280   540  1225
       D1           564   784   538   0  340   682  1170  2040
       W1           106  1882  1215   0  938  1950  2569  4565

In [18]:
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Neg']].describe().round(0).astype(int)

Chg Norm Neg                                       
                  count  mean   std   min   25%   50%  75% max
Symbol TF                                                     
WIN$N  M5         44176   -64    47  -390   -95   -55  -25  -5
       M15        20801  -116    87  -390  -170   -95  -45  -5
       H1          5401  -238   184  -915  -350  -190  -90  -5
       H3          1987  -397   325 -1410  -600  -305 -125  -5
       D1           547  -916   610 -2705 -1308  -825 -442  -5
       W1           112 -1674  1182 -4780 -2398 -1508 -686 -50

In [19]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm']].describe().round(0).astype(int)

HL Norm                                          
             count  mean   std   min   25%   50%   75%   max
Symbol TF                                                   
WIN$N  M5    88312   161    69    65   105   145   200   680
       M15   40514   295   125    95   195   265   370   725
       H1    10302   592   248   215   395   535   745  1375
       H3     3787   962   440   210   610   895  1255  2250
       D1     1062  1946   590  1000  1490  1815  2319  3805
       W1      217  4464  1268  2525  3385  4170  5360  8305

In [20]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Pos']].describe().round(0).astype(int)

HL Norm Pos                                          
                 count  mean   std   min   25%   50%   75%   max
Symbol TF                                                       
WIN$N  M5        45244   159    69    65   105   140   200   635
       M15       20452   292   124   115   195   265   365   725
       H1         5218   587   247   215   395   525   740  1325
       H3         1942   948   439   210   595   888  1225  2250
       D1          546  1915   582  1000  1480  1790  2270  3805
       W1          105  4535  1312  2525  3360  4375  5530  8305

In [21]:
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Neg']].describe().round(0).astype(int)

HL Norm Neg                                          
                 count  mean   std   min   25%   50%   75%   max
Symbol TF                                                       
WIN$N  M5        43068  -162    70  -680  -205  -145  -110   -65
       M15       20062  -298   126  -705  -375  -270  -200   -95
       H1         5084  -597   249 -1375  -750  -545  -400  -215
       H3         1845  -976   442 -2175 -1280  -905  -630  -210
       D1          516 -1979   596 -3635 -2356 -1835 -1504 -1045
       W1          112 -4397  1228 -7640 -5309 -4092 -3404 -2580

# Chart

In [22]:
# for SYMBOL, Asset in Src.groupby('Symbol', sort=0):

#     plt.title(f'{SYMBOL} - Candle Size Distribuition', fontweight='bold')

#     for TF, Data in Asset.groupby('TF', sort=0): 
#         # sns.kdeplot(Data['Change'], label=TF)
#         # sns.histplot(Data['Change'], label=TF, stat='count', bins=100, element='step', fill=False)
#         plt.hist(Data['Change'], bins=100, label=TF, histtype='step')


#     plt.ylabel('Frequency')
#     plt.xlabel('Candle Size')

#     plt.yscale('log', base=10)
#     plt.gca().yaxis.set_major_formatter(mpl.ticker.ScalarFormatter())

#     plt.legend()
#     plt.tight_layout()
#     plt.show()

# Snippets

In [23]:
def FORMAT_SOURCE(Raw):
    Src = pd.DataFrame()
    Src['Direct']       = (Src['Close'] - Src['Open']).apply(lambda x: -1 if (x <  0) else  +1)
    Src['Bull Sign']    =                Src['Direct'].apply(lambda x:  x if (x >= 0) else nan)
    Src['Bear Sign']    =                Src['Direct'].apply(lambda x:  x if (x <  0) else nan)

    Src['Change']       = Src['Close'] - Src['Open']
    Src['HL']           = Src['High']  - Src['Low']
    return Src 


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Psc']         = Calc.groupby(['Symbol','TF'], sort=0)['Change'] .expanding(20).rank(ascending=1, pct=1).mul(100)
    Calc['HL Psc']          = Calc.groupby(['Symbol','TF'], sort=0)['HL']     .expanding(20).rank(ascending=1, pct=1).mul(100)
    
    Calc['Chg Norm']        = Calc.apply(lambda x: x['Change'] if (5 < x['Chg Psc'] < 95) else nan, axis=1)
    Calc['HL Norm']         = Calc.apply(lambda x: x['HL']     if (5 < x['HL Psc']  < 95) else nan, axis=1)

    Calc['Chg Norm Abs']    = Calc['Chg Norm'].abs()
    Calc['Chg Norm Pos']    = Calc['Chg Norm'].apply(lambda x: x if (x >= 0) else nan)
    Calc['Chg Norm Neg']    = Calc['Chg Norm'].apply(lambda x: x if (x <  0) else nan)
    
    Calc['HL Norm Pos']     = Calc['HL Norm'] * Calc['Bull Sign']
    Calc['HL Norm Neg']     = Calc['HL Norm'] * Calc['Bear Sign']
    return Calc


Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['HL Norm Neg']].describe().round(0).astype(int)

Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Abs']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Pos']].describe().round(0).astype(int)
Calc.groupby(['Symbol','TF'], sort=0)[['Chg Norm Neg']].describe().round(0).astype(int)


Chg Norm Neg                                       
                  count  mean   std   min   25%   50%  75% max
Symbol TF                                                     
WIN$N  M5         44176   -64    47  -390   -95   -55  -25  -5
       M15        20801  -116    87  -390  -170   -95  -45  -5
       H1          5401  -238   184  -915  -350  -190  -90  -5
       H3          1987  -397   325 -1410  -600  -305 -125  -5
       D1           547  -916   610 -2705 -1308  -825 -442  -5
       W1           112 -1674  1182 -4780 -2398 -1508 -686 -50